# Chapter16. 코루틴
- 코루틴으로 작동하는 제너레이터의 동작과 상태
- 데커레이터를 이용해서 코루틴을 자동으로 기동하기
- 제너레이터 객체의 close()와 throw() 메서드를 통해 호출자가 코루틴을 제어한ㄴ 방법
- 종료할 때 코루틴이 값을 반환하는 방법
- 새로운 yield from 구문의 사용법과 의미
- 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?
(코루틴 기반 구조는 파이썬 2.5에 구현된 PEP342 - https://peps.python.org/pep-0342/) 에 설명되어 있다. 이때부터 yield 키워드를 표현식에 사용할 수 있게 되었으며, send()a메서드가 제너레이터 API에 추가되었다. 제너레이터의 호출자는 send()를 이용해서 제너레이터 함수 내부의 yield 표현식의 값이 될 데이터를 전송할 수 있다. 이렇게 제너레이터가 호출자에 데이터를 생성해주고 호출자로부터 데이터를 받으면서 호출자와 협업하는 프로시저인 코루틴이 된다.

최근 코루틴으로의 혁신적인 진화는 파이썬 3.3(2012)에서 구횐된 PEP380 [하위 제너레이터에 위임하기 위한 구문 제안서](https://www.python.org/dev/peps/pep-0380/)에 기술되어 있다. PEP380은 제너레이터 함수에 다음과 같이 두 가지 구문 변경을 정의해서 훨씬 더 유용하게 코루틴으로 사용할 수 있도록 만들었다.

- Procedure는 데이터베이스에서 일련의 쿼리들을 하나의 논리적인 작업으로 실행하기 위한 기술이며, 이는 데이터베이스의 특정 기능이나 작업을 자동화하고 자주 사용되는 쿼리들을 하나의 함수처럼 실행하기 위한 쿼리의 집합으로 구성되어 있다. 특히, 이 쿼리들은 데이터베이스 개발자가 일괄적인 작업을 수행하거나 특정한 목적을 위해 사용할 수 있도록 미리 구성되어 있다.

In [6]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine finished')

In [7]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x10793d4a0>

In [8]:
next(my_coro)

-> coroutine started


In [9]:
my_coro.send(42)

-> coroutine finished


StopIteration: 

In [10]:
my_coro = simple_coroutine()
my_coro.send(1211)

TypeError: can't send non-None value to a just-started generator

In [11]:
def simple_coro2(a):
    print('-> Started: a=', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield a + b
    print('-> Received: c=', c)

In [12]:
from inspect import getgeneratorstate

my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2)

'GEN_CREATED'

## 16.3 예제: 이동 평균을 계산하는 코루틴

더 복잡한 코루틴 예제를 만들어보자. 클로저를 생성해서 total과 counnt 변수를 보존하는 고급 함수를 구현했다. 

In [2]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:  # <1>
        term = yield average  # <2>
        total += term
        count += 1
        average = total/count

In [21]:
coro_avg = averager()
coro_avg

<generator object averager at 0x1084073c0>

In [22]:
next(coro_avg)
coro_avg.send(10)

10.0

In [23]:
coro_avg.send(30)

20.0

In [24]:
coro_avg.send(5)

15.0

In [25]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [32]:
coro_avg.close()
getgeneratorstate(coro_avg)

'GEN_CLOSED'

In [27]:
my_coro2 = simple_coro2(14)

In [29]:
my_coro2.send(None)

-> Started: a= 14


14

## 16.4 코루틴을 기동하기 위한 데커레이터
코루틴은 기동되기 전에는 할 수 있는 일이 많지 않다. `my_coro.send(x)`를 처음 호출하기 전에 반드시 `next(my_coro)`를 호출해야 한다. 코루틴을 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. 대표적으로 `@coroutine` 데커레이터가 널리 사용된다.

In [1]:
from functools import wraps

def coroutine(func):
    """데커레이터: 'func'를 기동해서 첫 번 째 'yield'까지 진행한다."""
    @wraps(func)
    def primer(*args, **kwargs):    # <1> primer로 치환되며, 실행하면 기동된 제너레이터 반환
        gen = func(*args, **kwargs)     # <2> 데커레이트된 함수를 호출해서 제너레이터 객체를 가져옴
        next(gen)   # <3> 제너레이터 기동
        return gen  # <4> 제너레이터 반환
    return primer


In [10]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:  # <1>
        term = yield average  # <2>
        total += term
        count += 1
        average = total/count

In [11]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [13]:
coro_avg.send(10)

10.0

In [14]:
coro_avg.send(30)

20.0

In [15]:
coro_avg.send(5)

15.0

## 16.5 코루틴 종료와 예외 처리
코루틴 안에서 발생한 예외를 처리하지 않으면, next()나 send()로 코루틴을 호출한 호출자에 예외가 전파된다.

In [16]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [17]:
coro_avg.send('spam')   # 비수치 값을 보내면 코루틴 내부에서 예외가 발생한다.

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [18]:
coro_avg.send(60)   # 예외를 처리하지 않으므로 코루틴 종료됨

StopIteration: 

In [2]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [3]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [4]:
exc_coro.send(11)

-> coroutine received: 11


In [5]:
exc_coro.send(22)

-> coroutine received: 22


In [6]:
exc_coro.close()

In [8]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [9]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [11]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [12]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [13]:
exc_coro.throw(ZeroDivisionError)   # 자신에게 던져진 예외를 처리할 수 없으면 코루틴이 종료된다.

ZeroDivisionError: 

In [14]:
# coro_finally_demo.py
class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine finished')

yield from 구조체가 파이썬 3.3에 추가된 이유 중 하나는 중첩된 코루틴에 예외를 던지는 것과 관련 있다. 그리고 코루틴에서 값을 좀 더 편리하게 반환할 수 있게 하기 위한 이유도 있다.

## 16.6 코루틴에서 값 반환하기
averger()는 코루틴을 변형해서 값을 반환한다. 이 코루틴은 활성화할 때마다 이동 평균을 생성하지는 않는다. 의미 있는 값을 생성하지는 않지만 최후에 어떤 의미 있는 값을 반환하는 (예를 들면 최종 합계를 반환하는 경우) 코루틴도 있음을 설명하기 위해서다.  
averger()가 반환하는 결과는 반환하는 결과는 namedtuple로서, 항목 수(count)와 평균(average)을 담고 있다. 그냥 average 값만 반환할 수도 있었지만, 튜플을 반환해서 누적된 데이터(항목 수)도 반환할 수 있음을 보여줌

In [15]:
# 16-13 coroaverager2.py
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [23]:
coro_avg = averager()
next(coro_avg)

In [24]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료됨.

StopIteration: Result(count=3, average=15.5)

return 문이 반환하는 값은 StopIteration 예외의 속성에 담겨 호출자에 밀반입됨에 주의하라. 실행이 완료되면 stopIteration 예외를 발생시키는 기존 제너레이터 객체의 작동 방식을 유지한다.

In [25]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료됨.
except StopIteration as exc:
    result = exc.value

result

Result(count=3, average=15.5)

## 16.7 yield from 사용하기
yield from은 완전히 새로운 언어 구성체이다. yield보다 훨씬 더 많은 일을 하므로 비슷한 키워드를 재사용한 것은 오해의 소지가 있다. 다른 언어에서는 이와 비슷한 구성체를 await이라고 하는데, 핵심을 잘 전달하므로 더 좋은 키워드이다.  
제너레이터 gen()이 yield from subgen()을 호출하고, subgen()이 이어받아 값을 생성하고 gen()의 호출자에 반환한다. 실질적으로 subgen()이 직접 호출자를 이끈다. 그러는 동안 gen()은 subgen()이 종료될 때까지 실행을 중단한다.

14장에서 yield from을 for루프 안의 yield에 대한 단축문으로 사용할 수 있다고 설명했다.

In [27]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [28]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

위의 코드는 동일하게 작동한다. 14.10절 파이썬 3.3의 새로운 구문: yield from에서 yield from을 처음 언급했을 때 아래 코드를 이용해서 사용법을 보여줬다.

In [29]:
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

더 유용한 예는 [python cookbook 4.14](https://frozenca.wordpress.com/category/programming/languages/python/study-python-cookbook/page/12/) 참고. yielf from x 표현식이 x 객체에 대해 첫 번째로 하는 일은 iter(x)를 호출해서 x의 반복자를 가져오는 것이다. 이는 모든 반복형이 x에 사용될 수 있다는 의미다.  
그러나 값을 생성하는 내포된 for 루프를 대체하는 게 yield from이 하는 일의 전부라면 yield from 구성자가 파이썬에 추가되지 않았을 것이다. Yield from의 진정한 가치는 단순한 반복형을 이요해서는 설명할 수 없고, 중첩된 제너레이터를 복잡하게 사용하는 예제가 필요하다.

In [1]:
!python coroaverager3.py

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


## 16.8 yield from의 의미
- 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자(즉, 클라이언트)에 바로 전달된다.
- send()를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 None이면 하위 제너레이터의 `__next__()`메서드가 호출된다. 전달된 값이 None이 아니면 하위 제너레이터의 send() 메서드가 호출된다. 호출된 메서드에서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
- 제너레이터나 하위 제너레이터에서 return expr 문을 실행하면, 제너레이터를 빠져나온 후 StopIteration(expr) 예외가 발생한다.
- 하위 제너레이터가 실행을 마친 후 발생한 StopIteration 예외의 첫 번째 인수가 yield from 표현식의 값이 된다.


## 16.9 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴
### 16.9. 1 이산 이벤트 시뮬레이션에 대해
이산 이벤트 시뮬레이션(DES; discrete evnt simulation)은 시스템을 일련의 이벤트로 모델링한다. DES에서 시뮬레이션 '시계'는 고정된 값만큼 진행하는 것이 아니라, 모델링된 다음 이벤트의 시뮬레이션된 시각으로 바로 진행한다. 예를 들어 상위 수준 관점에서 택시 운행을 시뮬레이션하는 경우, 승객을 태우는 이벤트 다음의 이벤트는 승객을 내리는 이벤트다. 운행 시간이 5분이든 50분이든 중요하지 않다. 승객을 내리는 이벤트가 발생하면 운행을 마친 시간으로 시계를 갱신한다. DES의 경우 1년치 택시 운행을 1초 안에 시뮬레이션할 수 있다.

### 16.9.2 택시 집단 시뮬레이션
taxi_sim.py 에서 아주 많은 태시를 생성해서 일정 횟수의 운행을 마친 후 집으로 돌아간다. 택시는 차고를 나와 승객을 찾으면서 '배회'한다. 이 상태는 승객을 태울 때까지 계속되며, 그러고 나서 운행이 시작된다. 승객이 내리고 나면, 택시는 다시 배회 상태로 들어간다.  
택시가 배회하고 운행하는 시간은 지수 분포를 이용해서 생성한다. 간결하게 출려하기 위해 분단위 시간을 사용하지만, 실수형 시간을 이용해서 시뮬레이션할 수도 있다. 각 상태의 상태 변화는 이벤트로 나타난다.

In [1]:
from taxi_sim import taxi_process
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)  # 택시 출발함

Event(time=0, proc=13, action='leave garage')

In [2]:
taxi.send(_.time + 7)   # 7분 뒤에 승객을 태움. 콘솔에서 _변수는 마지막 결과값에 바인딩 된다. (for 루프 시작 됨)

Event(time=7, proc=13, action='pick up passenger')

In [3]:
taxi.send(_.time + 23)  # 운행 중이므로 23을 보내면 첫 번째 승객을 23분간 태웠다는 의미임.

Event(time=30, proc=13, action='drop off passenger')

In [4]:
taxi.send(_.time + 5)   # 5분간 배회 후 승객을 태움

Event(time=35, proc=13, action='pick up passenger')

In [5]:
taxi.send(_.time + 48)  # 48분간 승객을 태우고 내려줌

Event(time=83, proc=13, action='drop off passenger')

In [6]:
taxi.send(_.time + 1)   # 운행을 마치고 집으로 감.

Event(time=84, proc=13, action='going home')

In [7]:
taxi.send(_.time + 10)  # 집으로 갔기 때문에 StopIteration 예외 발생함.

StopIteration: 

임의로 시뮬레이션 핵심 루프를 흉내냈다. 택시 코루틴이 생성한 Event 객체의 time 속성을 가져와서, 임의의 숫자를 더한 후, 그 합계를 코루틴을 다시 활성화하기 위해 taxi.send()를 호출하는 데 사용했다. 시뮬레이션에서는 Simulation.run() 메서드의 핵심 루프에서 taxi 코루틴을 다시 활성화시킨다. 시뮬레이션 '시계'는 sim_time 변수에 저장되어 이벤트가 생성될 때마다 갱신된다.

In [13]:
!python3 taxi_sim.py -s 3 -e 120 -t 2

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1    Event(time=5, proc=1, action='leave garage')
taxi: 1    Event(time=8, proc=1, action='pick up passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 0  Event(time=23, proc=0, action='pick up passenger')
taxi: 0  Event(time=25, proc=0, action='drop off passenger')
taxi: 0  Event(time=26, proc=0, action='going home')
taxi: 1    Event(time=27, proc=1, action='drop off passenger')
taxi: 1    Event(time=29, proc=1, action='pick up passenger')
taxi: 1    Event(time=35, proc=1, action='drop off passenger')
taxi: 1    Event(time=63, proc=1, action='pick up passenger')
taxi: 1    Event(time=76, proc=1, action='drop off passenger')
taxi: 1    Event(time=86, proc=1, action='pick up passenger')
taxi: 1    Event(time=99, proc=1, action='drop off passenger')
taxi: 1    Event(time=105, proc=1, action='going home')
*** end of events ***


## 16.11 읽을거리
브렛 슬랫킨의 [Effective Python]에 '여러 함수를 동시에 실행시키는 방법으로서의 코루틴'이라는 멋진 제목의 소절이 있는데 여기서 yield from으로 제너레이터를 실행하는 [최고의 예제](https://effectivepython.com/2015/03/10/consider-coroutines-to-run-many-functions-concurrently)를 볼 수 있다. 